In [57]:
#DEEP LEARNING


# Importing the libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tf

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, GRU, Bidirectional
from tensorflow.keras.optimizers import SGD
from tensorflow.random import set_seed
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import RepeatedKFold, train_test_split
from sklearn.metrics import mean_squared_error

In [58]:
import pandas as pd

# Load the dataset (replace 'delays.csv' with the correct path to your dataset file)
df = pd.read_csv('delays.csv')

# Display the columns of the DataFrame
print(df.columns)



Index(['MONTH', 'DAY_OF_MONTH', 'DAY_OF_WEEK', 'OP_UNIQUE_CARRIER', 'TAIL_NUM',
       'DEST', 'DEP_DELAY', 'CRS_ELAPSED_TIME', 'DISTANCE', 'CRS_DEP_M',
       'DEP_TIME_M', 'CRS_ARR_M', 'Temperature', 'Dew Point', 'Humidity',
       'Wind', 'Wind Speed', 'Wind Gust', 'Pressure', 'Condition', 'sch_dep',
       'sch_arr', 'TAXI_OUT'],
      dtype='object')


In [73]:
# Identify outliers
mean = df['TAXI_OUT'].mean()
std_dev = df['TAXI_OUT'].std()
threshold = 2
outliers = df[(df['TAXI_OUT'] - mean).abs() > threshold * std_dev]
df = df.drop(outliers.index)
df.head()


,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,OP_UNIQUE_CARRIER,TAIL_NUM,DEST,DEP_DELAY,CRS_ELAPSED_TIME,DISTANCE,CRS_DEP_M,...,Dew Point,Humidity,Wind,Wind Speed,Wind Gust,Pressure,Condition,sch_dep,sch_arr,TAXI_OUT
0,11,1,5,B6,N828JB,CHS,-1,124,636,324,...,34,58,W,25,38,29.86,Fair / Windy,9,17,14
1,11,1,5,B6,N992JB,LAX,-7,371,2475,340,...,34,58,W,25,38,29.86,Fair / Windy,9,17,15
2,11,1,5,B6,N959JB,FLL,40,181,1069,301,...,34,58,W,25,38,29.86,Fair / Windy,9,17,22
3,11,1,5,B6,N999JQ,MCO,-2,168,944,345,...,34,58,W,25,38,29.86,Fair / Windy,9,17,12
4,11,1,5,DL,N880DN,ATL,-4,139,760,360,...,32,58,W,24,35,29.91,Fair / Windy,9,17,13


In [74]:
date_column_name = 'Date'  # Corrected column name based on Kaggle dataset

# Ensure the column exists in the DataFrame
if date_column_name in df.columns:
    df[date_column_name] = pd.to_datetime(df[date_column_name], format='%d-%m-%Y')

    # Add a new column 'Wind' representing the month extracted from 'Date'
    df['Wind'] = df[date_column_name].dt.month

    # Check if all required columns are present in the DataFrame
    required_columns = ['Wind', 'Wind Speed', 'Wind Gust', 'Dew Point', 'Humidity', 'Condition', 'DISTANCE', 'Temperature', 'TAXI_OUT']
    missing_columns = [col for col in required_columns if col not in df.columns]

    if not missing_columns:
        # Select the required columns
        df = df[required_columns]

        # Display the first few rows of the DataFrame
        print(df.head())
    else:
        print(f"Missing columns: {missing_columns}")
else:
    print(f"Column '{date_column_name}' does not exist in the DataFrame.")


Column 'Date' does not exist in the DataFrame.


In [75]:
df.dropna(inplace=True)
df["Dew Point"] = df["Dew Point"].astype("int64")
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8345 entries, 0 to 8824
Data columns (total 23 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   MONTH              8345 non-null   int64  
 1   DAY_OF_MONTH       8345 non-null   int64  
 2   DAY_OF_WEEK        8345 non-null   int64  
 3   OP_UNIQUE_CARRIER  8345 non-null   object 
 4   TAIL_NUM           8345 non-null   object 
 5   DEST               8345 non-null   object 
 6   DEP_DELAY          8345 non-null   int64  
 7   CRS_ELAPSED_TIME   8345 non-null   int64  
 8   DISTANCE           8345 non-null   int64  
 9   CRS_DEP_M          8345 non-null   int64  
 10  DEP_TIME_M         8345 non-null   int64  
 11  CRS_ARR_M          8345 non-null   int64  
 12  Temperature        8345 non-null   int64  
 13  Dew Point          8345 non-null   int64  
 14  Humidity           8345 non-null   int64  
 15  Wind               8345 non-null   object 
 16  Wind Speed         8345 non-n

In [76]:
le = LabelEncoder()
df_le = df.copy()
for x in df_le.select_dtypes(["object_"]).columns:
    df_le[x] = le.fit_transform(df_le[x]).astype("str")
df_le

,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,OP_UNIQUE_CARRIER,TAIL_NUM,DEST,DEP_DELAY,CRS_ELAPSED_TIME,DISTANCE,CRS_DEP_M,...,Dew Point,Humidity,Wind,Wind Speed,Wind Gust,Pressure,Condition,sch_dep,sch_arr,TAXI_OUT
0,11,1,5,3,964,10,-1,124,636,324,...,34,58,15,25,38,29.86,3,9,17,14
1,11,1,5,3,1439,26,-7,371,2475,340,...,34,58,15,25,38,29.86,3,9,17,15
2,11,1,5,3,1357,19,40,181,1069,301,...,34,58,15,25,38,29.86,3,9,17,22
3,11,1,5,3,1453,28,-2,168,944,345,...,34,58,15,25,38,29.86,3,9,17,12
4,11,1,5,4,1097,1,-4,139,760,360,...,32,58,15,24,35,29.91,3,9,17,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8820,11,27,3,0,79,18,-2,137,509,1040,...,50,89,9,17,0,29.60,0,28,27,27
8821,11,27,3,1,17,26,20,384,2475,1020,...,50,89,9,17,0,29.60,0,28,27,27
8822,11,27,3,3,1348,58,-8,379,2521,1052,...,50,89,9,17,0,29.60,0,28,27,16
8823,11,27,3,3,605,49,4,192,1074,1041,...,50,89,9,17,0,29.60,0,28,27,17


In [77]:
from sklearn.preprocessing import LabelEncoder as LE
df=df.drop("TAIL_NUM",axis=1).copy()
df.sch_dep=LE().fit_transform(df.sch_dep)
df.Condition =LE().fit_transform(df.Condition)
df.Wind =LE().fit_transform(df.Wind.astype(str))
df.DEST=LE().fit_transform(df.DEST)
df.OP_UNIQUE_CARRIER=LE().fit_transform(df.OP_UNIQUE_CARRIER)

In [78]:
# Split features and target
X = df.drop(columns=['TAXI_OUT']).values
y = df['TAXI_OUT'].values
# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [79]:
# Define categorical columns (example: assuming 'Condition' is categorical)
cat_cols = ['Condition']

# Define embedding sizes
embedding_sizes = [(len(df[col].unique()), min(50, (len(df[col].unique()) + 1) // 2)) for col in cat_cols]
print("Embedding Sizes: ", embedding_sizes)
# Define embedding sizes
embedding_sizes = [(len(df[col].unique()), min(50, (len(df[col].unique()) + 1) // 2)) for col in cat_cols]

Embedding Sizes:  [(15, 8)]


In [80]:
# Build the model with embedding layers
input_layers = []
embedding_layers = []
for i, (input_dim, output_dim) in enumerate(embedding_sizes):
    input_layer = Input(shape=(1,))
    embedding_layer = Embedding(input_dim, output_dim)(input_layer)
    flatten_layer = Flatten()(embedding_layer)
    input_layers.append(input_layer)
    embedding_layers.append(flatten_layer)

In [81]:
model = Model(inputs=input_layers, outputs=output)

In [82]:
# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

In [95]:
# Importing necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Flatten, Dense, Concatenate

# Load the dataset (replace 'flight_take_off_data.csv' with the correct path to your dataset file)
df = pd.read_csv('delays.csv')

# Identify outliers
mean = df['TAXI_OUT'].mean()
std_dev = df['TAXI_OUT'].std()
threshold = 2
outliers = df[(df['TAXI_OUT'] - mean).abs() > threshold * std_dev]
df = df.drop(outliers.index)
df.head()

# Display the columns of the DataFrame
print(df.columns)

df.dropna(inplace=True)
df["Dew Point"] = df["Dew Point"].astype("int64")
df.info()

le = LabelEncoder()
df_le = df.copy()
for x in df_le.select_dtypes(["object_"]).columns:
    df_le[x] = le.fit_transform(df_le[x]).astype("str")
df_le

from sklearn.preprocessing import LabelEncoder as LE
df=df.drop("TAIL_NUM",axis=1).copy()
df.sch_dep=LE().fit_transform(df.sch_dep)
df.Condition =LE().fit_transform(df.Condition)
df.Wind =LE().fit_transform(df.Wind.astype(str))
df.DEST=LE().fit_transform(df.DEST)
df.OP_UNIQUE_CARRIER=LE().fit_transform(df.OP_UNIQUE_CARRIER)

# Data preprocessing
# Assuming the correct column names from the Kaggle dataset
date_column_name = 'Date'  # Corrected column name based on Kaggle dataset

# Ensure the column exists in the DataFrame
if date_column_name in df.columns:
    df[date_column_name] = pd.to_datetime(df[date_column_name], format='%d-%m-%Y')

    # Add a new column 'Wind' representing the month extracted from 'Date'
    df['Wind'] = df[date_column_name].dt.month

    # Check if all required columns are present in the DataFrame
    required_columns = ['Wind', 'Wind Speed', 'Wind Gust', 'Dew Point', 'Humidity', 'Condition', 'DISTANCE', 'Temperature', 'TAXI_OUT']
    missing_columns = [col for col in required_columns if col not in df.columns]

    if not missing_columns:
        # Select the required columns
        df = df[required_columns]

        # Display the first few rows of the DataFrame
        print(df.head())
    else:
        print(f"Missing columns: {missing_columns}")
else:
    print(f"Column '{date_column_name}' does not exist in the DataFrame.")

# Define categorical columns (example: assuming 'Condition' is categorical)
cat_cols = ['Condition']

# Define embedding sizes
embedding_sizes = [(len(df[col].unique()), min(50, (len(df[col].unique()) + 1) // 2)) for col in cat_cols]
print("Embedding Sizes: ", embedding_sizes)

# Example: Split data into train and test sets (assuming df is your DataFrame)
X = df.drop('TAXI_OUT', axis=1).values
y = df['TAXI_OUT'].values

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build the model with embedding layers
input_layers = []
embedding_layers = []
for i, (input_dim, output_dim) in enumerate(embedding_sizes):
    input_layer = Input(shape=(1,), name=f'input_{cat_cols[i]}')
    embedding_layer = Embedding(input_dim, output_dim, name=f'embedding_{cat_cols[i]}')(input_layer)
    flatten_layer = Flatten()(embedding_layer)
    input_layers.append(input_layer)
    embedding_layers.append(flatten_layer)

# Concatenate embedding layers
concatenated = Concatenate()(embedding_layers)
x = Dense(64, activation='relu')(concatenated)
x = Dense(32, activation='relu')(x)
output = Dense(1)(x)

model = Model(inputs=input_layers, outputs=output)

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

# Summary of the model
model.summary()

# Assuming categorical features are stored in cat_features_train and cat_features_test
cat_features_train = [X_train[:, i] for i in range(len(cat_cols))]
cat_features_test = [X_test[:, i] for i in range(len(cat_cols))]

# Fit the model (assuming continuous features are empty or not used here)
model.fit(cat_features_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

# Evaluate the model
loss, mae = model.evaluate(cat_features_test, y_test)
print(f'Test Loss: {loss}, Test MAE: {mae}')


Index(['MONTH', 'DAY_OF_MONTH', 'DAY_OF_WEEK', 'OP_UNIQUE_CARRIER', 'TAIL_NUM',
       'DEST', 'DEP_DELAY', 'CRS_ELAPSED_TIME', 'DISTANCE', 'CRS_DEP_M',
       'DEP_TIME_M', 'CRS_ARR_M', 'Temperature', 'Dew Point', 'Humidity',
       'Wind', 'Wind Speed', 'Wind Gust', 'Pressure', 'Condition', 'sch_dep',
       'sch_arr', 'TAXI_OUT'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
Index: 8345 entries, 0 to 8824
Data columns (total 23 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   MONTH              8345 non-null   int64  
 1   DAY_OF_MONTH       8345 non-null   int64  
 2   DAY_OF_WEEK        8345 non-null   int64  
 3   OP_UNIQUE_CARRIER  8345 non-null   object 
 4   TAIL_NUM           8345 non-null   object 
 5   DEST               8345 non-null   object 
 6   DEP_DELAY          8345 non-null   int64  
 7   CRS_ELAPSED_TIME   8345 non-null   int64  
 8   DISTANCE           8345 non-null   int64  
 9   CRS_DEP

Model: "functional_19"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_Condition (InputLayer)    │ (None, 1)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding_Condition (Embedding) │ (None, 1, 8)           │           120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_10 (Flatten)            │ (None, 8)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ concatenate_5 (Concatenate)     │ (None, 8)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_21 (Dense)                │ (None, 64)             │           576 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_22 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_23 (Dense)                │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,809 (10.97 KB)

 Trainable params: 2,809 (10.97 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
167/167 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 310.4628 - mae: 16.1956 - val_loss: 27.3554 - val_mae: 4.2304
Epoch 2/10
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 26.9531 - mae: 4.2519 - val_loss: 26.9055 - val_mae: 4.2336
Epoch 3/10
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 27.6784 - mae: 4.3240 - val_loss: 27.3545 - val_mae: 4.2303
Epoch 4/10
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 26.9713 - mae: 4.2519 - val_loss: 26.9684 - val_mae: 4.2762
Epoch 5/10
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 27.1038 - mae: 4.2783 - val_loss: 27.2177 - val_mae: 4.2248
Epoch 6/10
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 28.1068 - mae: 4.3603 - val_loss: 26.9397 - val_mae: 4.2704
Epoch 7/10
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 28.0184 - mae: 4.3718 - val_loss: 26.9556 - val_mae: 4.2203
Epoch 8/10
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 27.8208 - mae: 4.3446 - val_loss: 27.1182 - val_mae: 4.2200
Epoch 9/10
167/167 ━━━━━━━━━━━━━━━━━━━

In [96]:
# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

# Summary of the model
model.summary()

# Assuming categorical features are stored in cat_features_train and cat_features_test
cat_features_train = [X_train[:, i] for i in range(len(cat_cols))]
cat_features_test = [X_test[:, i] for i in range(len(cat_cols))]

# Fit the model (assuming continuous features are empty or not used here)
model.fit(cat_features_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

# Evaluate the model
loss, mae = model.evaluate(cat_features_test, y_test)
print(f'Test Loss: {loss}, Test MAE: {mae}')


Model: "functional_19"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_Condition (InputLayer)    │ (None, 1)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding_Condition (Embedding) │ (None, 1, 8)           │           120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_10 (Flatten)            │ (None, 8)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ concatenate_5 (Concatenate)     │ (None, 8)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_21 (Dense)                │ (None, 64)             │           576 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_22 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_23 (Dense)                │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,809 (10.97 KB)

 Trainable params: 2,809 (10.97 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
167/167 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 28.2123 - mae: 4.3811 - val_loss: 26.9201 - val_mae: 4.2287
Epoch 2/10
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 27.7515 - mae: 4.3302 - val_loss: 26.8916 - val_mae: 4.2528
Epoch 3/10
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 28.3340 - mae: 4.3768 - val_loss: 26.9091 - val_mae: 4.2323
Epoch 4/10
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 27.2954 - mae: 4.2903 - val_loss: 27.2887 - val_mae: 4.2278
Epoch 5/10
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 27.9816 - mae: 4.3483 - val_loss: 26.8893 - val_mae: 4.2499
Epoch 6/10
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 27.8105 - mae: 4.3367 - val_loss: 26.8952 - val_mae: 4.2555
Epoch 7/10
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 27.8830 - mae: 4.3548 - val_loss: 27.0553 - val_mae: 4.2164
Epoch 8/10
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 28.0465 - mae: 4.3420 - val_loss: 28.1943 - val_mae: 4.2537
Epoch 9/10
167/167 ━━━━━━━━━━━━━━━━━━━━ 

In [97]:
# Fit the model (assuming continuous features are empty or not used here)
model.fit(cat_features_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

# Evaluate the model
loss, mae = model.evaluate(cat_features_test, y_test)print(f'Test Loss: {loss}, Test MAE: {mae}')

Epoch 1/10
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 27.7644 - mae: 4.3210 - val_loss: 27.3495 - val_mae: 4.2301
Epoch 2/10
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 26.9661 - mae: 4.2777 - val_loss: 26.8989 - val_mae: 4.2366
Epoch 3/10
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 27.6428 - mae: 4.3342 - val_loss: 27.0396 - val_mae: 4.2154
Epoch 4/10
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 27.1203 - mae: 4.2847 - val_loss: 26.9123 - val_mae: 4.2630
Epoch 5/10
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 27.1556 - mae: 4.2866 - val_loss: 27.2466 - val_mae: 4.2260
Epoch 6/10
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 26.7928 - mae: 4.2386 - val_loss: 27.2336 - val_mae: 4.2255
Epoch 7/10
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 27.0720 - mae: 4.2708 - val_loss: 26.9433 - val_mae: 4.2229
Epoch 8/10
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 28.1077 - mae: 4.3707 - val_loss: 26.9081 - val_mae: 4.2615
Epoch 9/10
167/167 ━━━━━━━━━━━━━━━━━━━━ 